In [1]:
import pandas as pd

from sklearn.svm import SVR #SVR
from sklearn.tree import DecisionTreeRegressor #decisiontreeregressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split as holdout

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
#import library

from google.cloud import bigquery

client = bigquery.Client()
query = """
SELECT 
a.* except (loan_adm_pemerintahan_pertahanan_jaminan_sosial_wajib_112023,
simpanan_giro_nominal_112023,
simpanan_berjangka_nominal_112023,
simpanan_tabungan_nominal_112023,
loan_flat_dan_apartemen_112023,
loan_industri_pengolahan_112023,
loan_informasi_komunikasi_112023,
loan_jasa_lainnya_112023,
loan_jasa_pendidikan_112023,
loan_jasa_perusahaan_112023,
loan_kendaraan_bermotor_112023,
loan_kesehatan_dan_kegiatan_lain_112023,
loan_keuangan_dan_asuransi_112023,
loan_konstruksi_112023,
loan_lainnya_112023,
loan_pengadaan_air_pengelolaan_sampah_limbah_daur_ulang_112023,
loan_pengadaan_listrik_dan_gas_112023,
loan_penyedia_akm_makan_minum_112023,
loan_perdagangan_besar_ecer_reparasi_mobil_motor_112023,
loan_pertambangan_penggalian_112023,
loan_pertanian_kehutanan_perikanan_112023,
loan_real_estate_112023,
loan_ruko_rukan_112023,
loan_rumah_tinggal_112023,
loan_transportasi_pergudangan_112023),
b.jenis_kpi,
b.jenis_sektor_ekonomi,
b.real_2018,
b.real_2019,
b.real_2020,
b.real_2021,
b.real_2022,
b.real_2023,
b.pred_2024,
b.pred_2025,
b.pred_2026,
b.pred_2027,
b.pred_2028
FROM 
`ai4indonesia.geoplan_sbx.ai4_reff_kab`  as a inner join
`ai4indonesia.geoplan_sbx.ai4_data_pinjaman_umkm_kab`  as b
on a.bps_kab = b.kab_id
"""
query_job = client.query(query)
results = query_job.result()

df = results.to_dataframe()

df.head()

,kab,bps_kab,prov,total_pnddk,jmlh_kk,luas_wilayah_gis,luas_wil_dukcapil,pnddk_per_m,usia_20sd24,usia_25sd29,...,real_2019,real_2020,real_2021,real_2022,real_2023,pred_2024,pred_2025,pred_2026,pred_2027,pred_2028
0,PASER,6401,KALIMANTAN TIMUR,298997,98146,11170.941089,4148.0,26.765605,26656,23621,...,278305.47,17848.68,20374.88,41959.95,41968.52,41977.09,40893.98,41217.20,41282.27,41268.43
1,PASER,6401,KALIMANTAN TIMUR,298997,98146,11170.941089,4148.0,26.765605,26656,23621,...,167.23,93.54,2449.09,2054.32,1960.18,865.04,4270.22,2268.46,2208.15,1626.29
2,PASER,6401,KALIMANTAN TIMUR,298997,98146,11170.941089,4148.0,26.765605,26656,23621,...,7399.29,7525.87,7390.05,12277.41,15371.93,13517.63,14200.19,18443.83,20375.53,19510.81
3,PASER,6401,KALIMANTAN TIMUR,298997,98146,11170.941089,4148.0,26.765605,26656,23621,...,118356.11,99912.82,133076.60,116666.86,149008.76,128208.62,144065.35,163466.35,176088.45,156482.03
4,PASER,6401,KALIMANTAN TIMUR,298997,98146,11170.941089,4148.0,26.765605,26656,23621,...,514694.08,483451.29,527537.23,447307.62,455944.55,498781.72,484389.98,484078.52,508321.01,514115.24


In [2]:
df.columns

Index(['kab', 'bps_kab', 'prov', 'total_pnddk', 'jmlh_kk', 'luas_wilayah_gis',
       'luas_wil_dukcapil', 'pnddk_per_m', 'usia_20sd24', 'usia_25sd29',
       'usia_30sd34', 'usia_35sd39', 'usia_40sd44', 'lulus_sma', 'd1_d2', 'd3',
       's1', 's2', 's3', 'tdk_bekerja', 'asn', 'petani', 'peternak', 'nelayan',
       'pljr_mhswa', 'pedagang', 'perawat', 'popses_a1', 'popses_a2',
       'popses_b', 'popses_c1', 'popses_c2', 'popses_d', 'popses_e',
       'jmlh_kk_a1', 'jmlh_kk_a2', 'jmlh_kk_b', 'jmlh_kk_c1', 'jmlh_kk_c2',
       'jmlh_kk_d', 'jmlh_kk_e', 'mostses', 'avrg_income', 'jenis_kpi',
       'jenis_sektor_ekonomi', 'real_2018', 'real_2019', 'real_2020',
       'real_2021', 'real_2022', 'real_2023', 'pred_2024', 'pred_2025',
       'pred_2026', 'pred_2027', 'pred_2028'],
      dtype='object')

In [3]:
df = df.set_index('bps_kab')
df.fillna(0, inplace=True)

df.head()

,kab,prov,total_pnddk,jmlh_kk,luas_wilayah_gis,luas_wil_dukcapil,pnddk_per_m,usia_20sd24,usia_25sd29,usia_30sd34,...,real_2019,real_2020,real_2021,real_2022,real_2023,pred_2024,pred_2025,pred_2026,pred_2027,pred_2028
bps_kab,,,,,,,,,,,,,,,,,,,,,
6401,PASER,KALIMANTAN TIMUR,298997,98146,11170.941089,4148.0,26.765605,26656,23621,24217,...,278305.47,17848.68,20374.88,41959.95,41968.52,41977.09,40893.98,41217.20,41282.27,41268.43
6401,PASER,KALIMANTAN TIMUR,298997,98146,11170.941089,4148.0,26.765605,26656,23621,24217,...,167.23,93.54,2449.09,2054.32,1960.18,865.04,4270.22,2268.46,2208.15,1626.29
6401,PASER,KALIMANTAN TIMUR,298997,98146,11170.941089,4148.0,26.765605,26656,23621,24217,...,7399.29,7525.87,7390.05,12277.41,15371.93,13517.63,14200.19,18443.83,20375.53,19510.81
6401,PASER,KALIMANTAN TIMUR,298997,98146,11170.941089,4148.0,26.765605,26656,23621,24217,...,118356.11,99912.82,133076.60,116666.86,149008.76,128208.62,144065.35,163466.35,176088.45,156482.03
6401,PASER,KALIMANTAN TIMUR,298997,98146,11170.941089,4148.0,26.765605,26656,23621,24217,...,514694.08,483451.29,527537.23,447307.62,455944.55,498781.72,484389.98,484078.52,508321.01,514115.24


In [4]:
target = [
'real_2018',
'real_2019',
'real_2020',
'real_2021',
'real_2022',
'real_2023',
'pred_2024',
'pred_2025',
'pred_2026',
'pred_2027',
'pred_2028'
]

In [9]:
df_new = df.loc[df["jenis_kpi"] == "Jasa Keuangan Dan Asuransi"].copy()

x = df_new.drop([
    'kab',
    'prov',
    'jenis_kpi',
    'jenis_sektor_ekonomi',
'real_2018',
'real_2019',
'real_2020',
'real_2021',
'real_2022',
'real_2023',
'pred_2024',
'pred_2025',
'pred_2026',
'pred_2027',
'pred_2028'], axis = 1) #Prediktor
y = df_new['real_2018'] #Label


In [10]:
y_target = ['real_2018',
'real_2019',
'real_2020',
'real_2021',
'real_2022',
'real_2023',
'pred_2024',
'pred_2025',
'pred_2026',
'pred_2027',
'pred_2028']

In [11]:
y.head()

bps_kab
6401      6129.74
6402    199175.41
6403     11026.42
6409      8248.53
6171     35354.95
Name: real_2018, dtype: float64

In [12]:
X_train, X_test, y_train, y_test = holdout(x, y, test_size=0.25, random_state=22)


In [13]:

svr = SVR()
dt = DecisionTreeRegressor()
rf = RandomForestRegressor()
mlp = MLPRegressor(hidden_layer_sizes=(100,50,30,15,10, 10))
lr = LinearRegression()
knn = KNeighborsRegressor()


#train
svr.fit(X_train, y_train)
dt.fit(X_train, y_train)
rf.fit(X_train, y_train)
mlp.fit(X_train, y_train)
lr.fit(X_train, y_train)
knn.fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


KNeighborsRegressor()

In [14]:
import numpy as np

y_pred_svr = svr.predict(X_test)
y_pred_dt = dt.predict(X_test)
y_pred_rf = rf.predict(X_test)
y_pred_mlp = mlp.predict(X_test)
y_pred_lr = lr.predict(X_test)
y_pred_knn = knn.predict(X_test)

print("SVR")
print("MSE: ",mean_squared_error(y_test,y_pred_svr))
print("RMSE: ", np.sqrt(mean_squared_error(y_test,y_pred_svr)))
print("R2: ",r2_score(y_test,y_pred_svr))
print("")

print("Decission Tree Regression")
print("MSE: ",mean_squared_error(y_test,y_pred_dt))
print("RMSE: ", np.sqrt(mean_squared_error(y_test,y_pred_dt)))
print("R2: ",r2_score(y_test,y_pred_dt))
print("")

print("Random Forest Regression")
print("MSE: ",mean_squared_error(y_test,y_pred_rf))
print("RMSE: ", np.sqrt(mean_squared_error(y_test,y_pred_rf)))
print("R2: ",r2_score(y_test,y_pred_rf))
print("")

print("MLP Regression")
print("MSE: ",mean_squared_error(y_test,y_pred_mlp))
print("RMSE: ", np.sqrt(mean_squared_error(y_test,y_pred_mlp)))
print("R2: ",r2_score(y_test,y_pred_mlp))
print("")

print("Linear Regression")
print("MSE: ",mean_squared_error(y_test,y_pred_lr))
print("RMSE: ", np.sqrt(mean_squared_error(y_test,y_pred_lr)))
print("R2: ",r2_score(y_test,y_pred_lr))
print("")

print("KNN")
print("MSE: ",mean_squared_error(y_test,y_pred_knn))
print("RMSE: ", np.sqrt(mean_squared_error(y_test,y_pred_knn)))
print("R2: ",r2_score(y_test,y_pred_knn))
print("")

#LAG 1
# SVR
# MSE:  46354.59285912892
# RMSE:  215.30116780716477
# R2:  -6.705221132195093

# Decission Tree Regression
# MSE:  6037.452380952381
# RMSE:  77.70104491544744
# R2:  -0.0035662660593165363

# Random Forest Regression
# MSE:  6794.314012950828
# RMSE:  82.42762894170151
# R2:  -0.12937443050042874

# MLP Regression
# MSE:  4084.473084060486
# RMSE:  63.90988252266222
# R2:  0.3210647234713663

# Linear Regression
# MSE:  2394.832091799372
# RMSE:  48.93702168909927
# R2:  0.6019227070364079

# KNN
# MSE:  8284.283809523808
# RMSE:  91.01804112110855
# R2:  -0.3770423756763426

#LAG3
# SVR
# MSE:  43852.88402276194
# RMSE:  209.41080206799728
# R2:  -6.052630152114012

# Decission Tree Regression
# MSE:  4160.475
# RMSE:  64.50174416246432
# R2:  0.33089254935008694

# Random Forest Regression
# MSE:  3547.062980000001
# RMSE:  59.557224414843255
# R2:  0.42954439869426353

# MLP Regression
# MSE:  10022.044968423888
# RMSE:  100.11016416140716
# R2:  -0.6117931147575446

# Linear Regression
# MSE:  2102.168584776421
# RMSE:  45.8494120439556
# R2:  0.6619192129273492

# KNN
# MSE:  5969.379999999999
# RMSE:  77.26176285848
# R2:  0.03997581195402522

#REAL
# SVR
# MSE:  222978500152007.97
# RMSE:  14932464.63756094
# R2:  -0.09595512013717866

# Decission Tree Regression
# MSE:  165246308545011.62
# RMSE:  12854816.550422322
# R2:  0.18780269034811414

# Random Forest Regression
# MSE:  80419592683423.53
# RMSE:  8967697.178396666
# R2:  0.60473200644609

# MLP Regression
# MSE:  90245988161989.62
# RMSE:  9499788.848284451
# R2:  0.556434576739254

# Linear Regression
# MSE:  114980233776330.98
# RMSE:  10722883.650228187
# R2:  0.434864007804185

# KNN
# MSE:  113364343735800.47
# RMSE:  10647269.308879176
# R2:  0.4428062217947303

SVR
MSE:  6931965909.651606
RMSE:  83258.42846013613
R2:  -0.07845890576787617

Decission Tree Regression
MSE:  10567270505.523111
RMSE:  102797.23004791088
R2:  -0.6440310201860726

Random Forest Regression
MSE:  9541314322.141172
RMSE:  97679.65152549006
R2:  -0.4844151771024643

MLP Regression
MSE:  14209741136.263233
RMSE:  119204.61877068033
R2:  -1.2107180094065733

Linear Regression
MSE:  76516004803.15598
RMSE:  276615.26494963357
R2:  -10.904179548668393

KNN
MSE:  10508281156.47
RMSE:  102509.90760150942
R2:  -0.6348535963988029



In [15]:
import pickle
df_new = df.loc[df["jenis_sektor_ekonomi"] == "Pinjaman Berdasarkan Lapangan Usaha"].copy()

for xx in df_new['jenis_sektor_ekonomi'].unique():
    for yy in df_new['jenis_kpi'].unique():
        df_new2 = df_new.loc[df_new["jenis_kpi"] == yy].copy()
        for zz in y_target:
            x = df_new2.drop([
                'kab',
                'prov',
                'jenis_kpi',
                'jenis_sektor_ekonomi',
                'real_2018',
                'real_2019',
                'real_2020',
                'real_2021',
                'real_2022',
                'real_2023',
                'pred_2024',
                'pred_2025',
                'pred_2026',
                'pred_2027',
                'pred_2028'], axis = 1).copy()
            y = df_new2[zz] 
            print(xx+"_"+yy+"_"+zz, end=" ")
            #X_train, X_test, y_train, y_test = holdout(x, y, test_size=0.2, random_state=5)
            mlp = MLPRegressor(hidden_layer_sizes=(100,50,30,15,10, 10), max_iter=2500, random_state=22 )
            mlp.fit(x, y)
            print("MLP Regression Done", end=" ")
            
            rf = RandomForestRegressor(n_estimators=175, max_depth=7)
            rf.fit(x, y)
            print("RF Regression Done")

            print("")
            with open("model/umkm/my_model_mlp_umkm_"+xx+"_"+yy+"_"+zz+".pkl", "wb") as f:
              pickle.dump(mlp, f)
            with open("model/umkm/my_model_rf_umkm_"+xx+"_"+yy+"_"+zz+".pkl", "wb") as f:
              pickle.dump(rf, f)

Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_real_2018 MLP Regression Done RF Regression Done

Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_real_2019 MLP Regression Done RF Regression Done

Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_real_2020 MLP Regression Done RF Regression Done

Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_real_2021 MLP Regression Done RF Regression Done

Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_real_2022 MLP Regression Done RF Regression Done

Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_real_2023 MLP Regression Done RF Regression Done

Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_pred_2024 MLP Regression Done RF Regression Done

Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_pred_2025 MLP Regression Done RF Regression Done

Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_pred_2026 MLP Regression Done RF Regression Done

Pinjaman Berdasarkan Lapangan Usaha_Industri P

In [25]:
import pickle
df_new = df.loc[df["jenis_sektor_ekonomi"] == "kelompok usaha"].copy()

for xx in df_new['jenis_sektor_ekonomi'].unique():
    for yy in df_new['jenis_kpi'].unique():
        df_new2 = df_new.loc[df_new["jenis_kpi"] == yy].copy()
        for zz in y_target:
            x = df_new2.drop([
                'kab',
                'prov',
                'jenis_kpi',
                'jenis_sektor_ekonomi',
                'real_2018',
                'real_2019',
                'real_2020',
                'real_2021',
                'real_2022',
                'real_2023',
                'pred_2024',
                'pred_2025',
                'pred_2026',
                'pred_2027',
                'pred_2028'], axis = 1).copy()
            y = df_new2[zz] 
            print(xx+"_"+yy+"_"+zz, end=" ")
            #X_train, X_test, y_train, y_test = holdout(x, y, test_size=0.2, random_state=5)
            mlp = MLPRegressor(hidden_layer_sizes=(100,50,30,15,10, 10), max_iter=1000, random_state=22 )
            mlp.fit(x, y)
            print("MLP Regression Done", end=" ")
            
            rf = RandomForestRegressor(n_estimators=175, max_depth=7)
            rf.fit(x, y)
            print("RF Regression Done")

            print("")
            with open("model/umkm/my_model_mlp_umkm_"+xx+"_"+yy+"_"+zz+".pkl", "wb") as f:
              pickle.dump(mlp, f)
            with open("model/umkm/my_model_rf_umkm_"+xx+"_"+yy+"_"+zz+".pkl", "wb") as f:
              pickle.dump(rf, f)

kelompok usaha_Kecil_real_2018 MLP Regression Done RF Regression Done

kelompok usaha_Kecil_real_2019 MLP Regression Done RF Regression Done

kelompok usaha_Kecil_real_2020 MLP Regression Done RF Regression Done

kelompok usaha_Kecil_real_2021 MLP Regression Done RF Regression Done

kelompok usaha_Kecil_real_2022 MLP Regression Done RF Regression Done

kelompok usaha_Kecil_real_2023 MLP Regression Done RF Regression Done

kelompok usaha_Kecil_pred_2024 MLP Regression Done RF Regression Done

kelompok usaha_Kecil_pred_2025 MLP Regression Done RF Regression Done

kelompok usaha_Kecil_pred_2026 MLP Regression Done RF Regression Done

kelompok usaha_Kecil_pred_2027 MLP Regression Done RF Regression Done

kelompok usaha_Kecil_pred_2028 MLP Regression Done RF Regression Done

kelompok usaha_Mikro_real_2018 MLP Regression Done RF Regression Done

kelompok usaha_Mikro_real_2019 MLP Regression Done RF Regression Done

kelompok usaha_Mikro_real_2020 MLP Regression Done RF Regression Done

kelomp

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Regression Done RF Regression Done

kelompok usaha_Mikro_pred_2028 MLP Regression Done RF Regression Done

kelompok usaha_Menengah_real_2018 MLP Regression Done RF Regression Done

kelompok usaha_Menengah_real_2019 MLP Regression Done RF Regression Done

kelompok usaha_Menengah_real_2020 MLP Regression Done RF Regression Done

kelompok usaha_Menengah_real_2021 MLP Regression Done RF Regression Done

kelompok usaha_Menengah_real_2022 MLP Regression Done RF Regression Done

kelompok usaha_Menengah_real_2023 MLP Regression Done RF Regression Done

kelompok usaha_Menengah_pred_2024 MLP Regression Done RF Regression Done

kelompok usaha_Menengah_pred_2025 MLP Regression Done RF Regression Done

kelompok usaha_Menengah_pred_2026 MLP Regression Done RF Regression Done

kelompok usaha_Menengah_pred_2027 MLP Regression Done RF Regression Done

kelompok usaha_Menengah_pred_2028 MLP Regression Done RF Regression Done



In [26]:
#Get data kec
client = bigquery.Client()
query = """
SELECT * FROM `ai4indonesia.geoplan_sbx.ai4_reff_kec`
"""
query_job = client.query(query)
results = query_job.result()

df_kec = results.to_dataframe()

df_kec.head()

,bps_kec_id,kab_name,bps_kab_id,prov_name,kec_name,kmdgr_kec_id,total_pnddk,jmlh_kk,luas_wilayah_gis,luas_wil_dukcapil,...,popses_e,jmlh_kk_a1,jmlh_kk_a2,jmlh_kk_b,jmlh_kk_c1,jmlh_kk_c2,jmlh_kk_d,jmlh_kk_e,mostses,avrg_income
0,1605040,MUSI RAWAS,1605,SUMATERA SELATAN,TUGUMULYO,16.05.01,49017,16172,58.200867,57,...,162,760,1391,5285,5588,1890,1091,167,4,1149531
1,1605080,MUSI RAWAS,1605,SUMATERA SELATAN,MUARA LAKITAN,16.05.02,43544,13197,1747.876038,622,...,142,619,1135,4315,4562,1539,890,137,4,1149531
2,1605070,MUSI RAWAS,1605,SUMATERA SELATAN,MUARA KELINGI,16.05.03,45222,13928,651.289693,742,...,149,655,1199,4551,4815,1626,938,144,4,1149531
3,1605060,MUSI RAWAS,1605,SUMATERA SELATAN,JAYALOKA,16.05.08,16763,5433,157.124750,156,...,54,256,468,1774,1879,634,364,58,4,1149531
4,1605050,MUSI RAWAS,1605,SUMATERA SELATAN,MUARA BELITI,16.05.09,28212,8759,172.835738,173,...,92,410,755,2862,3026,1025,590,91,4,1149531


In [27]:
# df_kec = df_kec.set_index('bps_kec')
df_kec.fillna(0, inplace=True)
df_kec["id_nama"] = df_kec["bps_kec_id"]+"-"+df_kec["kec_name"]
df_kec.head()

,bps_kec_id,kab_name,bps_kab_id,prov_name,kec_name,kmdgr_kec_id,total_pnddk,jmlh_kk,luas_wilayah_gis,luas_wil_dukcapil,...,jmlh_kk_a1,jmlh_kk_a2,jmlh_kk_b,jmlh_kk_c1,jmlh_kk_c2,jmlh_kk_d,jmlh_kk_e,mostses,avrg_income,id_nama
0,1605040,MUSI RAWAS,1605,SUMATERA SELATAN,TUGUMULYO,16.05.01,49017,16172,58.200867,57,...,760,1391,5285,5588,1890,1091,167,4,1149531,1605040-TUGUMULYO
1,1605080,MUSI RAWAS,1605,SUMATERA SELATAN,MUARA LAKITAN,16.05.02,43544,13197,1747.876038,622,...,619,1135,4315,4562,1539,890,137,4,1149531,1605080-MUARA LAKITAN
2,1605070,MUSI RAWAS,1605,SUMATERA SELATAN,MUARA KELINGI,16.05.03,45222,13928,651.289693,742,...,655,1199,4551,4815,1626,938,144,4,1149531,1605070-MUARA KELINGI
3,1605060,MUSI RAWAS,1605,SUMATERA SELATAN,JAYALOKA,16.05.08,16763,5433,157.124750,156,...,256,468,1774,1879,634,364,58,4,1149531,1605060-JAYALOKA
4,1605050,MUSI RAWAS,1605,SUMATERA SELATAN,MUARA BELITI,16.05.09,28212,8759,172.835738,173,...,410,755,2862,3026,1025,590,91,4,1149531,1605050-MUARA BELITI


In [28]:
# df_twr = df_kec.loc[df_kec["id_nama"] == "6101010-SELAKAU"]
df_twr = df_kec.copy()
x = df_twr.drop([
  'kab_name','bps_kab_id','prov_name','kec_name','kmdgr_kec_id','bps_kec_id','id_nama'], axis = 1).copy()
x1 = df_twr.drop([
  'kab_name','bps_kab_id','prov_name','kec_name','kmdgr_kec_id','bps_kec_id','id_nama'], axis = 1).copy()

In [29]:
x1.head()


,total_pnddk,jmlh_kk,luas_wilayah_gis,luas_wil_dukcapil,pnddk_per_m,usia_20sd24,usia_25sd29,usia_30sd34,usia_35sd39,usia_40sd44,...,popses_e,jmlh_kk_a1,jmlh_kk_a2,jmlh_kk_b,jmlh_kk_c1,jmlh_kk_c2,jmlh_kk_d,jmlh_kk_e,mostses,avrg_income
0,49017,16172,58.200867,57,842.203943,3971,3608,3381,3472,3911,...,162,760,1391,5285,5588,1890,1091,167,4,1149531
1,43544,13197,1747.876038,622,24.912522,3659,3186,3346,3729,3512,...,142,619,1135,4315,4562,1539,890,137,4,1149531
2,45222,13928,651.289693,742,69.434540,3851,3368,3318,3903,3748,...,149,655,1199,4551,4815,1626,938,144,4,1149531
3,16763,5433,157.124750,156,106.685929,1375,1092,1289,1420,1456,...,54,256,468,1774,1879,634,364,58,4,1149531
4,28212,8759,172.835738,173,163.230130,2383,2186,2296,2455,2305,...,92,410,755,2862,3026,1025,590,91,4,1149531


In [30]:
with open("model/umkm/my_model_mlp_umkm_Pinjaman Berdasarkan Lapangan Usaha_Pengadaan Air, Pengelolaan Sampah, Limbah Dan Daur Ulang_pred_2028.pkl", "rb") as f:
  mlp = pickle.load(f)

with open("model/umkm/my_model_rf_umkm_Pinjaman Berdasarkan Lapangan Usaha_Pengadaan Air, Pengelolaan Sampah, Limbah Dan Daur Ulang_pred_2028.pkl", "rb") as f:
     rf = pickle.load(f)

In [31]:
import numpy as np

y_pred_mlp = mlp.predict(x)
y_pred_rf = rf.predict(x)
# if y_pred_mlp<0:
#     y_pred = y_pred_rf
# else:
#     y_pred = y_pred_mlp
    
# x1['id_nama'] = "6101010-SELAKAU"
x1['jenis_kpi'] = "Nominal"
x1['jenis_sektor_ekonomi'] = "Tabungan"
x1['pred_year'] = "pred_2018"
x1['value_mlp'] = y_pred_mlp
x1['value_rf'] = y_pred_rf
x1['id_nama'] = df_kec['id_nama']
x1['value'] = np.where(x1['value_mlp'] < 0, x1['value_rf'], x1['value_mlp'])
# x1.head()

df_result = x1.copy()

In [32]:
empty_df = pd.DataFrame(columns=df_result.columns)

# Assign the empty DataFrame to the existing DataFrame (effectively deleting values)
df_result = empty_df.copy()

df_result.head()

,total_pnddk,jmlh_kk,luas_wilayah_gis,luas_wil_dukcapil,pnddk_per_m,usia_20sd24,usia_25sd29,usia_30sd34,usia_35sd39,usia_40sd44,...,jmlh_kk_e,mostses,avrg_income,jenis_kpi,jenis_sektor_ekonomi,pred_year,value_mlp,value_rf,id_nama,value


In [33]:
i=0
#df_new = df.loc[df["jenis_sektor_ekonomi"] == "Pinjaman Berdasarkan Lapangan Usaha"].copy()

x = df_twr.drop(['kab_name','bps_kab_id','prov_name','kec_name','kmdgr_kec_id','bps_kec_id','id_nama'], axis = 1).copy()
x1 = df_twr.drop(['kab_name','bps_kab_id','prov_name','kec_name','kmdgr_kec_id','bps_kec_id','id_nama'], axis = 1).copy()
    
for xx in df['jenis_sektor_ekonomi'].unique():
    x = df_twr.drop(['kab_name','bps_kab_id','prov_name','kec_name','kmdgr_kec_id','bps_kec_id','id_nama'], axis = 1).copy()
    x1 = df_twr.drop(['kab_name','bps_kab_id','prov_name','kec_name','kmdgr_kec_id','bps_kec_id','id_nama'], axis = 1).copy()
    
    df_new = df.loc[df["jenis_sektor_ekonomi"] == xx].copy()
    
    for yy in df_new['jenis_kpi'].unique():
        for zz in y_target:
            print(xx+"_"+yy+"_"+zz,end=", ")
            with open("model/umkm/my_model_mlp_umkm_"+xx+"_"+yy+"_"+zz+".pkl", "rb") as f:
                mlp = pickle.load(f)
            with open("model/umkm/my_model_rf_umkm_"+xx+"_"+yy+"_"+zz+".pkl", "rb") as f:
                rf = pickle.load(f)

            y_pred_mlp = mlp.predict(x)
            y_pred_rf = rf.predict(x)
           
            x1['jenis_sektor_ekonomi'] = xx
            x1['jenis_kpi'] = yy
            x1['pred_year'] = zz 
            x1['value_mlp'] = y_pred_mlp
            x1['value_rf'] = y_pred_rf
            x1['id_nama'] = df_kec['id_nama']
            x1['value'] = np.where(x1['value_mlp'] < 0, x1['value_rf'], x1['value_mlp'])    
            
            # Concatenate DataFrames
            df_result= pd.concat([df_result, x1], ignore_index=True)
            # df_result = df_result.append(x1, ignore_index=True)
            print(str(i)+"_pred_done")
            i=i+1

Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_real_2018, 0_pred_done
Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_real_2019, 1_pred_done
Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_real_2020, 2_pred_done
Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_real_2021, 3_pred_done
Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_real_2022, 4_pred_done
Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_real_2023, 5_pred_done
Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_pred_2024, 6_pred_done
Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_pred_2025, 7_pred_done
Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_pred_2026, 8_pred_done
Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_pred_2027, 9_pred_done
Pinjaman Berdasarkan Lapangan Usaha_Industri Pengolahan_pred_2028, 10_pred_done
Pinjaman Berdasarkan Lapangan Usaha_Jasa Pendidikan_real_2018, 11_pred_done
Pinjaman Berdasarkan Lapangan Usaha_Jasa Pendidikan_re

In [34]:
df_result

,total_pnddk,jmlh_kk,luas_wilayah_gis,luas_wil_dukcapil,pnddk_per_m,usia_20sd24,usia_25sd29,usia_30sd34,usia_35sd39,usia_40sd44,...,jmlh_kk_e,mostses,avrg_income,jenis_kpi,jenis_sektor_ekonomi,pred_year,value_mlp,value_rf,id_nama,value
0,49017,16172,58.200867,57,842.203943,3971,3608,3381,3472,3911,...,167,4,1149531,Industri Pengolahan,Pinjaman Berdasarkan Lapangan Usaha,real_2018,5422.699037,14538.380654,1605040-TUGUMULYO,5422.699037
1,43544,13197,1747.876038,622,24.912522,3659,3186,3346,3729,3512,...,137,4,1149531,Industri Pengolahan,Pinjaman Berdasarkan Lapangan Usaha,real_2018,5040.417151,6446.843962,1605080-MUARA LAKITAN,5040.417151
2,45222,13928,651.289693,742,69.434540,3851,3368,3318,3903,3748,...,144,4,1149531,Industri Pengolahan,Pinjaman Berdasarkan Lapangan Usaha,real_2018,5070.701219,7468.774841,1605070-MUARA KELINGI,5070.701219
3,16763,5433,157.124750,156,106.685929,1375,1092,1289,1420,1456,...,58,4,1149531,Industri Pengolahan,Pinjaman Berdasarkan Lapangan Usaha,real_2018,3671.340364,5221.013576,1605060-JAYALOKA,3671.340364
4,28212,8759,172.835738,173,163.230130,2383,2186,2296,2455,2305,...,91,4,1149531,Industri Pengolahan,Pinjaman Berdasarkan Lapangan Usaha,real_2018,4231.352700,5406.768335,1605050-MUARA BELITI,4231.352700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1598515,2565,743,51.330128,38,49.970652,339,322,279,237,165,...,0,7,1160643,Menengah,kelompok usaha,pred_2028,137945.495285,392871.821001,9110073-AYAMARU TENGAH,137945.495285
1598516,1539,500,74.917416,57,20.542620,202,157,136,117,98,...,0,7,1160643,Menengah,kelompok usaha,pred_2028,137850.842976,392871.821001,9110074-AYAMARU BARAT,137850.842976
1598517,1462,488,123.924255,91,11.797529,155,148,125,121,82,...,0,7,1160643,Menengah,kelompok usaha,pred_2028,137861.050853,391922.776944,9110021-AITINYO TENGAH,137861.050853
1598518,1534,539,29.401490,25,52.174226,155,129,142,125,67,...,0,7,1160643,Menengah,kelompok usaha,pred_2028,137964.929595,392871.821001,9110051-AITINYO RAYA,137964.929595


In [35]:
import pandas as pd
from google.cloud import bigquery

table_id = 'ai4indonesia.geoplan_sbx.ai4_data_pinjaman_umkm_kec_result'

# Write DataFrame to BigQuery
df_result.to_gbq(destination_table=table_id, project_id='ai4indonesia')

100%|██████████| 1/1 [00:00<00:00, 8525.01it/s]
